# Sentiment Analysis

## Importing the required libraries

In [1]:
import numpy as np
import pandas as pd

In [2]:
# Decompress the file
import gzip

# Visualizations
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
import seaborn as sns
import matplotlib.colors as colors
%matplotlib inline


# Datetime
from datetime import datetime

# text preprocessing

import nltk
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer, WordNetLemmatizer
from nltk.tokenize import word_tokenize, sent_tokenize, regexp_tokenize
from nltk.tokenize import RegexpTokenizer
from nltk.tokenize.toktok import ToktokTokenizer
import gensim
import re


In [3]:
pip install xgboost

Note: you may need to restart the kernel to use updated packages.


In [4]:
## Modeling
from sklearn.model_selection import cross_validate
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_predict
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import learning_curve
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.decomposition import TruncatedSVD
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer, HashingVectorizer
from sklearn.pipeline import Pipeline
from sklearn.naive_bayes import MultinomialNB
from sklearn.naive_bayes import GaussianNB 
from sklearn.svm import SVC
from sklearn import metrics
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from  sklearn.metrics import precision_recall_fscore_support
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier, GradientBoostingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import f1_score
from sklearn.metrics import recall_score
from gensim.models import Word2Vec
from tqdm import tqdm
import xgboost as xgb
from xgboost import XGBClassifier
from sklearn.dummy import DummyClassifier

## Warnings
import warnings
from scipy import stats
warnings.filterwarnings('ignore')

In [5]:
data=pd.read_csv('/users/STSC/Desktop/datacleaned6.csv')

In [6]:
data

,reviewer_id,product_id,reviewer_name,rating,unix_review_time,categories,review_text,pos_feedback,neg_feedback,rating_class,help_prop,review_time,clean_text
0,A30TL5EWN6DFXT,120401325X,christina,4,1400630400,['Cell Phones & Accessories'],Looks Good They look good and stick good! I ju...,0,0,good,0.000000,2014-05-21,look good look good stick good like rounded sh...
1,ASY55RVNIL0UD,120401325X,emily l.,5,1389657600,['Cell Phones & Accessories'],Really great product. These stickers work like...,0,0,good,0.000000,2014-01-14,great product sticker work like review say sti...
2,A2TMXE2AFO7ONB,120401325X,Erica,5,1403740800,['Cell Phones & Accessories'],LOVE LOVE LOVE These are awesome and make my p...,0,0,good,0.000000,2014-06-26,love love love awesome phone look stylish far ...
3,AWJ0WZQYMYFQ4,120401325X,JM,4,1382313600,['Cell Phones & Accessories'],Cute! Item arrived in great time and was in pe...,4,0,good,1.000000,2013-10-21,cute item arrive great time perfect condition ...
4,ATX7CZYFXI1KW,120401325X,patrice m rogoza,5,1359849600,['Cell Phones & Accessories'],leopard home button sticker for iphone 4s awes...,2,1,good,0.666667,2013-02-03,leopard home button sticker iphone awesome sta...
...,...,...,...,...,...,...,...,...,...,...,...,...,...
190991,A1YMNTFLNDYQ1F,B00LORXVUE,eyeused2loveher,5,1405900800,['Cell Phones & Accessories'],This works just perfect! Works great just like...,0,0,good,0.000000,2014-07-21,work perfect work great like original need ext...
190992,A15TX8B2L8B20S,B00LORXVUE,Jon Davidson,5,1405900800,['Cell Phones & Accessories'],Great replacement cable. Apple certified Great...,0,0,good,0.000000,2014-07-21,great replacement cable apple certify great pr...
190993,A3JI7QRZO1QG8X,B00LORXVUE,Joyce M. Davidson,5,1405900800,['Cell Phones & Accessories'],"Real quality This is a great cable, just as go...",0,0,good,0.000000,2014-07-21,real quality great cable good expensive apple ...
190994,A1NHB2VC68YQNM,B00LORXVUE,Nurse Farrugia,5,1405814400,['Cell Phones & Accessories'],I really like it becasue it works well with my...,0,0,good,0.000000,2014-07-20,like becasue work life proof like becasue work...


In [7]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 190996 entries, 0 to 190995
Data columns (total 13 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   reviewer_id       190996 non-null  object 
 1   product_id        190996 non-null  object 
 2   reviewer_name     190990 non-null  object 
 3   rating            190996 non-null  int64  
 4   unix_review_time  190996 non-null  int64  
 5   categories        190996 non-null  object 
 6   review_text       190996 non-null  object 
 7   pos_feedback      190996 non-null  int64  
 8   neg_feedback      190996 non-null  int64  
 9   rating_class      190996 non-null  object 
 10  help_prop         190996 non-null  float64
 11  review_time       190996 non-null  object 
 12  clean_text        190987 non-null  object 
dtypes: float64(1), int64(4), object(8)
memory usage: 18.9+ MB


## Data Preprocessing

In [8]:
data['rating_class'] = data['rating_class'].apply(lambda x: 0 if x == 'bad' else 1)

In [9]:
data.head()

,reviewer_id,product_id,reviewer_name,rating,unix_review_time,categories,review_text,pos_feedback,neg_feedback,rating_class,help_prop,review_time,clean_text
0,A30TL5EWN6DFXT,120401325X,christina,4,1400630400,['Cell Phones & Accessories'],Looks Good They look good and stick good! I ju...,0,0,1,0.000000,2014-05-21,look good look good stick good like rounded sh...
1,ASY55RVNIL0UD,120401325X,emily l.,5,1389657600,['Cell Phones & Accessories'],Really great product. These stickers work like...,0,0,1,0.000000,2014-01-14,great product sticker work like review say sti...
2,A2TMXE2AFO7ONB,120401325X,Erica,5,1403740800,['Cell Phones & Accessories'],LOVE LOVE LOVE These are awesome and make my p...,0,0,1,0.000000,2014-06-26,love love love awesome phone look stylish far ...
3,AWJ0WZQYMYFQ4,120401325X,JM,4,1382313600,['Cell Phones & Accessories'],Cute! Item arrived in great time and was in pe...,4,0,1,1.000000,2013-10-21,cute item arrive great time perfect condition ...
4,ATX7CZYFXI1KW,120401325X,patrice m rogoza,5,1359849600,['Cell Phones & Accessories'],leopard home button sticker for iphone 4s awes...,2,1,1,0.666667,2013-02-03,leopard home button sticker iphone awesome sta...


In [10]:
data['rating_class'].value_counts(normalize=True)*100

1    87.439004
0    12.560996
Name: rating_class, dtype: float64

Checking and dropping null values in clean_text Column

In [11]:
data['clean_text'].isnull().sum()

9

In [12]:
data.shape

(190996, 13)

In [13]:
data.dropna(subset='clean_text',inplace=True)
data.shape

(190987, 13)

### Train test split

In [14]:
# Splitting the Data Set into Train and Test Sets
X = data['clean_text']
y = data['rating_class']

In [15]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, stratify=y, random_state=2)

In [16]:
y_train=y_train.astype(np.uint8)
y_test=y_test.astype(np.uint8)

In [17]:
y_train

118818    1
184559    1
107041    1
78547     1
115754    1
         ..
86197     1
77133     1
75141     1
56405     1
79686     1
Name: rating_class, Length: 133690, dtype: uint8

In [18]:
print ('Train data Shape\t\t:{}\nTest data Shape \t\t:{}'.format(X_train.shape, X_test.shape))

Train data Shape		:(133690,)
Test data Shape 		:(57297,)


### Evaluation metrics

As the data is imbalanced (87 : 13), accuracy alone would'nt be enough to define the performance of the model.F1 score (which is harmonic average of precision and recall) is taken as the evaluation metrics for my Sentiment Analysis.

### NLP techniques to represent the clean text

As Machine Learning models can only take numerical values as inputs, we have to give a numerical representation to the text of our data. It is important to give a compact representation to our text in a correct dimensional space where the text is correctly classified

Some techniques in NLP to convert text to vector like Sentence Embedding is used to give a compact and Semantic representation thereby helping the model to classify correctly

### Sentence Embedding

In [19]:
#! pip install -user sentence-transform
#conda install -c conda-forge  sentence-transformers

In [20]:
#conda install pytorch torchvision cudatoolkit=10.0 -c pytorch

In [21]:
pip install sentence-transformers

Note: you may need to restart the kernel to use updated packages.


In [22]:
from sentence_transformers import SentenceTransformer
model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')

In [23]:
X_train=pd.DataFrame(X_train)
X_test=pd.DataFrame(X_test)
y_train=pd.DataFrame(y_train)
y_test=pd.DataFrame(y_test)

In [24]:
X_train['embeddings'] = X_train['clean_text'].apply(lambda x: model.encode(x))
X_test['embeddings'] = X_test['clean_text'].apply(lambda x: model.encode(x))

In [25]:
# Use the pandas `str.split` method to separate the array values into separate columns
X_train = pd.concat([X_train, pd.DataFrame(X_train['embeddings'].to_list(), index=X_train.index)], axis=1)

# Drop the original 'array_column'
X_train = X_train.drop(['embeddings'], axis=1)

In [26]:
# Use the pandas `str.split` method to separate the array values into separate columns
X_test = pd.concat([X_test, pd.DataFrame(X_test['embeddings'].to_list(), index=X_test.index)], axis=1)

# Drop the original 'array_column'
X_test = X_test.drop(['embeddings'], axis=1)

In [27]:
# Drop 'meaningful text' column

# X_train= X_train.drop(['clean_text'], axis=1)
X_test = X_test.drop(['clean_text'], axis=1)

In [28]:
X_test

,0,1,2,3,4,5,6,7,8,9,...,374,375,376,377,378,379,380,381,382,383
128426,-0.156219,0.132607,0.110814,-0.006796,0.055928,0.060973,0.089696,0.033085,-0.013454,0.008128,...,-0.034049,-0.062355,0.015428,0.009733,0.082623,0.072030,-0.034101,-0.056785,0.088663,0.062091
113741,-0.144971,0.085611,0.013527,-0.044792,0.028799,-0.033410,0.163164,-0.007703,0.054795,-0.023424,...,-0.047448,-0.035451,-0.013367,0.029177,0.070079,0.073259,0.072106,-0.070108,0.052988,0.083292
187122,-0.144203,0.071986,0.090844,-0.028498,-0.001849,0.016347,0.083961,0.076645,-0.023970,0.017016,...,-0.008716,-0.006318,0.019318,0.003037,0.060866,0.027765,0.072653,-0.115694,0.022138,0.099877
26696,0.010459,0.014937,0.031641,0.029845,0.046285,-0.040171,-0.036844,-0.034344,0.002391,-0.022451,...,-0.031968,-0.097583,0.021487,0.072431,-0.064887,-0.064604,0.099824,0.046481,0.046782,0.076168
21050,-0.039230,0.118317,0.056768,-0.034366,-0.032894,-0.074490,0.028159,0.023163,0.068850,-0.045877,...,-0.019876,-0.080959,-0.010038,0.002993,0.050169,-0.010004,-0.038015,-0.002873,0.067013,0.081945
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
118183,-0.067558,0.035119,0.045801,-0.034960,-0.043834,0.050184,0.094537,0.046963,-0.060854,-0.003236,...,0.072020,0.058910,0.026479,0.027365,0.014462,-0.004201,-0.030829,-0.052480,0.009092,0.105029
138646,-0.043065,-0.050395,0.030053,-0.074690,-0.116687,-0.018272,0.048663,-0.082951,-0.098882,-0.026309,...,0.057330,0.031348,0.002581,0.037472,-0.040107,0.012154,-0.001395,0.020935,0.077257,0.030098
26637,-0.125259,0.117507,0.144841,-0.004498,0.061985,-0.002368,0.062293,0.074033,-0.036438,-0.043860,...,0.004785,-0.007832,0.088023,0.006242,0.091545,0.003266,0.010872,-0.078393,0.031170,0.105769
7294,-0.127881,0.103354,0.057913,-0.070364,0.017815,-0.041611,0.077141,0.075036,-0.053498,0.001037,...,0.064435,0.020606,0.077374,0.009993,0.104747,0.005858,0.027618,0.006956,0.031436,0.084427


In [29]:
X_test.to_csv('testing_data_embeddings.csv')

In [30]:
X_train=pd.read_csv("training_data_embeddings.csv")
X_train

FileNotFoundError: [Errno 2] No such file or directory: 'training_data_embeddings.csv'

In [ ]:
X_train.drop('Unnamed: 0',axis=1,inplace=True)

     1. Logistic Regression  

In [ ]:
logreg = LogisticRegression()
logreg.fit(X_train, y_train)
y_val_pred = logreg.predict(X_test)

score = f1_score(y_test, y_val_pred, average = 'weighted')
score

    2. Decision Tree

In [ ]:
from sklearn.model_selection import GridSearchCV

param_grid = {"criterion": ["gini", "entropy"],
              "max_depth": [0, 200]}
dtclf3 = DecisionTreeClassifier()
dtclf_grid = GridSearchCV(dtclf3, param_grid, cv=3)
dtclf_grid.fit(X_train,y_train)
y_val_pred = dtclf_grid.predict(X_test)
score = f1_score(y_test, y_val_pred, average = 'weighted')
score

    3. Random Forest

In [ ]:
param_grid = {"n_estimators" : [20, 100],
              "max_depth" : [1,50],
              "max_features" : [30, 50]}
clf3 = RandomForestClassifier()
clf_grid = GridSearchCV(clf3, param_grid, cv=2)
clf_grid.fit(X_train,y_train)
y_val_pred = clf_grid.predict(X_test)
score = f1_score(y_test, y_val_pred, average = 'weighted')
score

    4. Gaussian Naive Bayes

In [ ]:
naive=GaussianNB()
naive.fit(X_train,y_train)
y_val_pred = naive.predict(X_test)
score = f1_score(y_test, y_val_pred, average = 'weighted')
score

     5. XG Boost

In [ ]:
xgb=XGBClassifier()
xgb.fit(X_train,y_train)
y_val_pred = xgb.predict(X_test)
score = f1_score(y_test, y_val_pred, average = 'weighted')
score

    6. AdaBoost

In [ ]:
clf_adaboost =  AdaBoostClassifier()
clf_adaboost.fit(X_train, y_train)
#print(clf_adaboost.best_params_)
y_val_pred = clf_adaboost.predict(X_test)
score = f1_score(y_test, y_val_pred, average = 'weighted')
score

    7. GradientBoost

In [ ]:
clf_gradient =  GradientBoostingClassifier()
clf_gradient.fit(X_train, y_train)
#print(clf_adaboost.best_params_)
y_val_pred = clf_gradient.predict(X_test)
score = f1_score(y_test, y_val_pred, average = 'weighted')
score

#### Visual Representation of the performance of the models in terms of Average F1 Score : 

In [ ]:
x = list(['Logistic Regression', 'Decision Tree', 'Random Forest', 'Gaussian Naive Bayes', 'XG Boost', 'AdaBoost', 'GradientBoost'])
y = list([0.8897452878278732,0.8218927850695935,0.8460581778722911,0.8053369909229872,0.8826017932341885,0.8628009871988357,0.8613825703362123])
performance=pd.DataFrame({'Model':x,'Avg F1 Score':y})
performance['Avg F1 Score']=performance['Avg F1 Score'].apply(lambda x: round(x,3))

In [ ]:
performance

In [ ]:
plt.figure(figsize=(15,10))
# Plot the bar chart using Seaborn
ax = sns.barplot(x='Model',y='Avg F1 Score', data=performance, palette='bright')

# Add values on top of each bar
for p in ax.patches:
    height = p.get_height()
    ax.text(p.get_x() + p.get_width() / 2., height, f'{height}', ha='center', va='bottom')
    if height == average_value:
        ax.annotate(f'Avg: {average_value:.2f}', (p.get_x() + p.get_width() / 2., height/2), ha='center', va='center', color='black')
    
average_value=performance['Avg F1 Score'].mean()    
plt.axhline(y=average_value, color='black', linestyle='--', label='Average')
# ax.text(len(performance) - 0.5, average_value + 1, f'Average: {average_value:.2f}', ha='right', va='bottom', color='black')

# Add labels and title
plt.xlabel('Models')
plt.ylabel('Average F1 Scores')
plt.title('Model Performance of different models w.r.to Average F1 Score')

# Show the plot
plt.show()